# Prelimnary Evaluation

In [2]:
# --- One Time Setup ---
import os

# Ensure setup is only done once
try:
    if SETUP_IS_DONE:
        print("Skipping Setup")
except:
    # Perform Setup
    
    # Path to NaxRiscv repository
    path_results = os.getcwd()
    path_nax_repo = os.getcwd() + "/../"
    path_benchmarker = path_nax_repo + "/ext/NaxSoftware/baremetal/benchmarker/"
    from pathlib import Path
    path_errors = path_results + "/errors/"
    Path(path_errors).mkdir(parents=True, exist_ok=True)
    
    # Fix because /bin is automatically added to the start of the PATH by default & messes up some verilator stuff
    # TODO: Is this really required?
    os.environ["PATH"] = os.environ["PATH"].split(":",1)[1]

SETUP_IS_DONE = True

# Presets to use for benchmark configurations
with_all_cores = [0, 1, 2, 3, 4, 5, 6, 7]
with_some_cores = [0, 3, 7]
with_min_cores = [0]

with_all_prios = [0, 1, 2, 3, 4, 5, 6, 7]
with_some_prios = [0, 1, 2]
with_min_prios = [0]
with_no_prios = ["none"]

with_all_coworkers = [0, 1, 2, 3, 4, 5, 6, 7]
with_some_coworkers = [0, 1, 4, 7]
with_min_coworkers = [0, 7]

with_standard_memconf = [ # (mem-resp-time, core-max-trans, down-pending-max=4, probe-count=1)
    (0, 2, 4, 1), 
    (0, 4, 4, 1), 
    (70, 2, 4, 1), 
    (281, 4, 4, 1),
]
with_asic_memconf = [ # (mem-resp-time, core-max-trans, down-pending-max=4, probe-count=1)
    (281, 4, 4, 1),
]
with_fpga_memconf = [ # (mem-resp-time, core-max-trans, down-pending-max=4, probe-count=1)
    (70, 2, 4, 1),
]
with_debug_memconf = [ # (mem-resp-time, core-max-trans, down-pending-max=4, probe-count=1)
    (0, 4, 4, 1),
]

In [6]:
# ------------ CONFIG ---------------

bench_configs = [ # ("BENCH_LATENCY" or "BENCH_BANDWIDTH", cores, prios, coworkers, memconf)
    # Q0
    ("BENCH_BANDWIDTH", with_min_prios, with_no_prios, with_some_coworkers, [ 
        # (mem-resp-time, core-max-trans, down-pending-max=4, probe-count=1)
        (70, 4, 4, 1),
        (281, 4, 4, 1),
        (281, 2, 4, 1),
        (281, 8, 4, 1),
        (281, 4, 2, 1),
        (281, 4, 8, 1),
        (281, 4, 4, 4),
        (281, 4, 4, 8),
    ]),

    # Q1
    # ("BENCH_BANDWIDTH", [ 0 ], with_no_prios, with_some_coworkers, with_asic_memconf),
    
    # Q3
    # ("BENCH_LATENCY", [ 0 ], with_no_prios, with_some_coworkers, with_debug_memconf),
    
    ]

bench_name = "var" # BENCH_NAME

# Timeout in seconds
timeout = 300 # 5 min

# ------------ ------ ---------------

def get_bench_num():
    bench_list = []
    for (bench, cores, prios, workers, memconf) in bench_configs:
        for main_core in cores:
            for main_prio in prios:
                for worker_num in workers:
                    for (mem_resp_time, core_max_transactions, down_pending_max, probe_count) in memconf:
                        bench_id = f"{bench_name}_{bench}_{main_core}main_{main_prio}prio_{worker_num}cores_{mem_resp_time}resp_{core_max_transactions}trans_{down_pending_max}dp_{probe_count}probes"
                        if bench_id in bench_list:
                            continue
                        bench_list.append(bench_id)
    return len(bench_list)

print(f"Benchmark is expected to finish in less than {int(get_bench_num() * timeout // 3600 )} h {int((get_bench_num() * timeout) % 3600 // 60)} min")

Benchmark is expected to finish in less than 2 h 40 min


In [20]:
import numpy as np
import pandas as pd
import os
import re
import subprocess
import time
import datetime
import shutil
import signal
from datetime import datetime, timedelta
        
print("Starting benchmarks...")
num_bench = get_bench_num()
bench_ctr = 0
start_time = time.time()
finished_before = (datetime.now() + timedelta(seconds=(timeout * num_bench))).strftime("%B %d %H:%M")
print(f"Benchmark is expected to finish before {finished_before}")

completed_ids = []
data_lines = []
data_header = ["Benchmark", "MainCore", "MainPrio", "WorkerNum", "MemMaxTransaction", "MemResponseTime", "DownPendingMax", "ProbeCount", "BufferSize", "Cycles"]
for (bench, cores, prios, workers, memconf) in bench_configs:
    for main_core in cores:
        for main_prio in prios:
            for worker_num in workers:
                
                # Compile the benchmarker
                print(f"  Compiling {bench} for core {main_core} with prio {main_prio} and {worker_num} coworkers")
                os.chdir(path_benchmarker)
                subprocess.run(["make", "clean"], check=True)
                subprocess.run(["make", f"WORKER_NUM={worker_num}", f"MAIN_CORE={main_core}", f"MAIN_PRIO={main_prio}", f"BENCH={bench}"], check=True, capture_output=True)
                
                # Run with different configurations
                for (mem_resp_time, core_max_transactions, down_pending_max, probe_count) in memconf:
                    max_remaining_time = (num_bench - bench_ctr) * timeout
                    bench_id = f"{bench_name}_{bench}_{main_core}main_{main_prio}prio_{worker_num}cores_{mem_resp_time}resp_{core_max_transactions}trans_{down_pending_max}dp_{probe_count}probes"
                    
                    if bench_id in completed_ids:
                        print(f"    Skipping benchmark with memconf={(mem_resp_time, core_max_transactions, down_pending_max, probe_count)}, already done")
                        continue
                    
                    bench_ctr += 1
                    print(f"    Running benchmark [{bench_ctr}/{num_bench}] with memconf={(mem_resp_time, core_max_transactions, down_pending_max, probe_count)}, max {int(max_remaining_time // 60)} min remaining")
                    completed_ids.append(bench_id)
                    
                    proc = None
                    try: 
                        os.chdir(path_nax_repo)
                        proc = subprocess.Popen(["sbt", f"runMain naxriscv.platform.rtccidemo.SocSim \
                                                --load-elf {path_benchmarker}/build/benchmarker.elf \
                                                --xlen 32 \
                                                --nax-count 8 \
                                                --core-max-trans {core_max_transactions} \
                                                --mem-resp-time {mem_resp_time} \
                                                --down-pending-max {down_pending_max} \
                                                --probe-count {probe_count} \
                                                --no-rvls \
                                                --trace"], stdout=subprocess.PIPE, preexec_fn=os.setsid)
                        (stdout_data, stderr_data) = proc.communicate(timeout=timeout)
                        
                        file_name = f"bench_{bench_id}"
                        try: # Save Waveform
                            shutil.copy(path_nax_repo + "/simWorkspace/SocDemo/test.fst",
                                        path_nax_repo + f"/waveforms/bench/{file_name}.fst")
                        except:
                            pass
                        
                        if proc.returncode == 0:
                            # Read Result
                            buffer_size = re.search(rb"BufferSize: [^=]*= (\d*)", stdout_data).group(1)
                            cycles = re.search(rb"Cycles: [^=]*= (\d*)", stdout_data).group(1)
                            print("    " + stdout_data.splitlines()[-1].decode("utf-8"))
                            data_lines += [[bench, main_core, main_prio, worker_num, core_max_transactions, mem_resp_time, down_pending_max, probe_count, int(buffer_size), int(cycles)]]
                            
                            # Save Progress
                            df = pd.DataFrame(data_lines, columns=data_header)
                            df.to_csv(path_results + f"/benchmark_{bench_name}_results_partial.csv", index=False)
                        else:
                            print(f"    [ERROR] simulation crashed, error dumped in errors/bench_{bench_name}_{mem_resp_time}resp_{core_max_transactions}trans_{down_pending_max}dp_{probe_count}probes_{worker_num}cores_{main_core}mc.txt")
                            with open(path_errors + f"bench_{bench_name}_{mem_resp_time}resp_{core_max_transactions}trans_{down_pending_max}dp_{probe_count}probes_{worker_num}cores_{main_core}mc.txt", "wb") as binary_file:
                                binary_file.write(stdout_data)
                                
                    except subprocess.TimeoutExpired:
                        print(f"    [ERROR] simulation timed out after {timeout} seconds")
                        
                        if proc.pid:
                            os.killpg(os.getpgid(proc.pid), signal.SIGTERM)
            
elapsed_time = time.time() - start_time

df = pd.DataFrame(data_lines, columns=data_header)
df.to_csv(path_results + f"/benchmark_{bench_name}_results.csv", index=False)

print(f"Benchmark completed in {int(elapsed_time // 60)} min {round(elapsed_time % 60, 2)} s")
print("Result saved to " + path_results + f"/benchmark_{bench_name}_results.csv")

Starting benchmarks...
Benchmark is expected to finish before February 28 16:40
  Compiling BENCH_BANDWIDTH for core 0 with prio none and 0 coworkers
    Running benchmark [1/2] with memconf=(0, 4, 4, 1), max 10 min remaining
    [success] Total time: 18 s, completed Feb 28, 2024, 4:31:18 PM
  Compiling BENCH_BANDWIDTH for core 0 with prio none and 7 coworkers
    Running benchmark [2/2] with memconf=(0, 4, 4, 1), max 5 min remaining
    [success] Total time: 30 s, completed Feb 28, 2024, 4:31:51 PM
Benchmark completed in 0 min 54.47 s
Result saved to /home/julianpritzi/NaxRiscv/_evaluation/benchmark_all_results.csv
